# Imports

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import h5py
import json
import os
from pathlib import Path
from quantify_core.data.handling import set_datadir, get_tuids_containing, extract_parameter_from_snapshot, load_snapshot, load_quantities_of_interest, load_dataset, to_gridded_dataset, load_processed_dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from lmfit.models import GaussianModel
from matplotlib.colors import PowerNorm, ListedColormap
from matplotlib.legend_handler import HandlerTuple
from scipy.stats import norm
from scipy.signal import find_peaks
import math
from scipy.optimize import minimize_scalar
from typing import List, Union
import xarray as xr
import re
import itertools
from SSRO_functions import *

%load_ext autoreload
%autoreload 2

# Set quantify data directory
set_datadir("O:\FPA_RO_Data")

In [ ]:
from matplotlib.colors import PowerNorm, ListedColormap
# Choose colormap
cmap_blue = plt.cm.Blues
cmap_red = plt.cm.Reds
cmap_green = plt.cm.Greens
cmap_purple = plt.cm.Purples
cmap_orange = plt.cm.Oranges

# Get the colormap colors
cmap_blue = cmap_blue(np.arange(plt.cm.Blues.N))
cmap_red = cmap_red(np.arange(plt.cm.Reds.N))
cmap_green = cmap_green(np.arange(plt.cm.Greens.N))
cmap_purple = cmap_purple(np.arange(plt.cm.Purples.N))
cmap_orange = cmap_orange(np.arange(plt.cm.Oranges.N))

# Set alpha
cmap_blue[:,-1] = np.linspace(0, 1, plt.cm.Blues.N)
cmap_red[:,-1] = np.linspace(0, 1, plt.cm.Reds.N)
cmap_green[:,-1] = np.linspace(0, 1, plt.cm.Greens.N)
cmap_purple[:,-1] = np.linspace(0, 1, plt.cm.Purples.N)
cmap_orange[:,-1] = np.linspace(0, 1, plt.cm.Oranges.N)

# Create new colormap
cmap_blue = ListedColormap(cmap_blue)
cmap_red = ListedColormap(cmap_red)
cmap_green = ListedColormap(cmap_green)
cmap_purple = ListedColormap(cmap_purple)
cmap_orange = ListedColormap(cmap_orange)

# Fidelity vs Flux Pulse Amplitude
performed for readout frequencies of 5.1755, 5.176, 5.1765, and 5.177 GHz, and readout amplitudes (100, 150, 200, 300, 400 mV)

## 100 mV

In [ ]:
tuid_ssro_100 = get_tuids_containing('SSRO FX8', t_start='20231116-142858', t_stop='20231116-151907')

In [ ]:
freq_ro_100 = []
flux_amp_100 = []

fid_ass_raw_100 = []
fid_discr_100 = []

# Loop over all data folders to get the readout frequency and flux pulse amplitude
for tuid in tuid_ssro_100:
    try:
        freq = get_RO_freq(tuid)
        flux_amp = get_flux_pulse_amp(tuid)
        fidelities = [get_raw_ass_fidelity(tuid), get_fid_discr(tuid)]
        
        freq_ro_100.append(freq)
        flux_amp_100.append(flux_amp)
        fid_ass_raw_100.append(fidelities[0])
        fid_discr_100.append(fidelities[1])
    except:
        print(tuid)

In [ ]:
idx_100_resfreq1 = np.argwhere(np.array(freq_ro_100) == 5.1755e9)[:,0]
idx_100_resfreq2 = np.argwhere(np.array(freq_ro_100) == 5.176e9)[:,0]
idx_100_resfreq3 = np.argwhere(np.array(freq_ro_100) == 5.1765e9)[:,0]
idx_100_resfreq4 = np.argwhere(np.array(freq_ro_100) == 5.177e9)[:,0]

fid_ass_raw_100_1 = [fid_ass_raw_100[i] for i in idx_100_resfreq1]
fid_ass_raw_100_2 = [fid_ass_raw_100[i] for i in idx_100_resfreq2]
fid_ass_raw_100_3 = [fid_ass_raw_100[i] for i in idx_100_resfreq3]
fid_ass_raw_100_4 = [fid_ass_raw_100[i] for i in idx_100_resfreq4]

fid_discr_100_1 = [fid_discr_100[i] for i in idx_100_resfreq1]
fid_discr_100_2 = [fid_discr_100[i] for i in idx_100_resfreq2]
fid_discr_100_3 = [fid_discr_100[i] for i in idx_100_resfreq3]
fid_discr_100_4 = [fid_discr_100[i] for i in idx_100_resfreq4]

flux_amp_100_1 = [flux_amp_100[i] for i in idx_100_resfreq1]
flux_amp_100_2 = [flux_amp_100[i] for i in idx_100_resfreq2]
flux_amp_100_3 = [flux_amp_100[i] for i in idx_100_resfreq3]
flux_amp_100_4 = [flux_amp_100[i] for i in idx_100_resfreq4]

In [ ]:
plt.xlabel('Flux Pulse Amplitude (V)')
plt.ylabel('Raw Assignment Fidelity')
plt.title('Readout Amplitude: 100 mV')
plt.xlim(flux_amp_100_1[0], flux_amp_100_1[-1])
plt.plot(flux_amp_100_1, fid_ass_raw_100_1, marker='o', linestyle=' ', color=cmap_purple(0.4), label='5.1755 GHz')
plt.plot(flux_amp_100_2, fid_ass_raw_100_2, marker='o', linestyle=' ', color=cmap_purple(0.6), label='5.176 GHz')
plt.plot(flux_amp_100_3, fid_ass_raw_100_3, marker='o', linestyle=' ', color=cmap_purple(0.8), label='5.1765 GHz')
plt.plot(flux_amp_100_4, fid_ass_raw_100_4, marker='o', linestyle=' ', color=cmap_purple(1.0), label='5.177 GHz')
plt.legend(loc='best', frameon=False)
plt.show()

## 150 mV

In [ ]:
tuid_ssro_150_1 = get_tuids_containing('SSRO FX8', t_start='20231116-121406', t_stop='20231116-123430')
tuid_ssro_150_2 = get_tuids_containing('SSRO FX8', t_start='20231116-135120', t_stop='20231116-135438')
tuid_ssro_150_3 = get_tuids_containing('SSRO FX8', t_start='20231116-135737', t_stop='20231116-142213')
tuid_ssro_150 = np.concatenate((tuid_ssro_150_1, tuid_ssro_150_2, tuid_ssro_150_3))

In [ ]:
freq_ro_150 = []
flux_amp_150 = []

fid_ass_raw_150 = []
fid_discr_150 = []

# Loop over all data folders to get the readout frequency and flux pulse amplitude
for tuid in tuid_ssro_150:
    try:
        freq = get_RO_freq(tuid)
        flux_amp = get_flux_pulse_amp(tuid)
        fidelities = [get_raw_ass_fidelity(tuid), get_fid_discr(tuid)]
        
        freq_ro_150.append(freq)
        flux_amp_150.append(flux_amp)
        fid_ass_raw_150.append(fidelities[0])
        fid_discr_150.append(fidelities[1])
    except:
        print(tuid)

In [ ]:
idx_150_resfreq1 = np.argwhere(np.array(freq_ro_150) == 5.1755e9)[:,0]
idx_150_resfreq2 = np.argwhere(np.array(freq_ro_150) == 5.176e9)[:,0]
idx_150_resfreq3 = np.argwhere(np.array(freq_ro_150) == 5.1765e9)[:,0]
idx_150_resfreq4 = np.argwhere(np.array(freq_ro_150) == 5.177e9)[:,0]

fid_ass_raw_150_1 = [fid_ass_raw_150[i] for i in idx_150_resfreq1]
fid_ass_raw_150_2 = [fid_ass_raw_150[i] for i in idx_150_resfreq2]
fid_ass_raw_150_3 = [fid_ass_raw_150[i] for i in idx_150_resfreq3]
fid_ass_raw_150_4 = [fid_ass_raw_150[i] for i in idx_150_resfreq4]

fid_discr_150_1 = [fid_discr_150[i] for i in idx_150_resfreq1]
fid_discr_150_2 = [fid_discr_150[i] for i in idx_150_resfreq2]
fid_discr_150_3 = [fid_discr_150[i] for i in idx_150_resfreq3]
fid_discr_150_4 = [fid_discr_150[i] for i in idx_150_resfreq4]

flux_amp_150_1 = [flux_amp_150[i] for i in idx_150_resfreq1]
flux_amp_150_2 = [flux_amp_150[i] for i in idx_150_resfreq2]
flux_amp_150_3 = [flux_amp_150[i] for i in idx_150_resfreq3]
flux_amp_150_4 = [flux_amp_150[i] for i in idx_150_resfreq4]

In [ ]:
plt.xlabel('Flux Pulse Amplitude (V)')
plt.ylabel('Raw Assignment Fidelity')
plt.title('Readout Amplitude: 150 mV')
plt.xlim(flux_amp_150_1[0], flux_amp_150_1[-1])
plt.plot(flux_amp_150_1, fid_ass_raw_150_1, marker='o', linestyle=' ', color=cmap_green(0.4), label='5.1755 GHz')
plt.plot(flux_amp_150_2, fid_ass_raw_150_2, marker='o', linestyle=' ', color=cmap_green(0.6), label='5.176 GHz')
plt.plot(flux_amp_150_3, fid_ass_raw_150_3, marker='o', linestyle=' ', color=cmap_green(0.8), label='5.1765 GHz')
plt.plot(flux_amp_150_4, fid_ass_raw_150_4, marker='o', linestyle=' ', color=cmap_green(1.0), label='5.177 GHz')
plt.legend(loc='best', frameon=False)
plt.show()

## 200 mV

In [ ]:
tuid_ssro_200_1 = get_tuids_containing('SSRO FX8', t_start='20231115-114040', t_stop='20231115-120638')
tuid_ssro_200_2 = get_tuids_containing('SSRO FX8', t_start='20231115-125706', t_stop='20231115-132243')
tuid_ssro_200 = np.concatenate((tuid_ssro_200_1, tuid_ssro_200_2))

In [ ]:
freq_ro_200 = []
flux_amp_200 = []

fid_ass_raw_200 = []
fid_discr_200 = []

# Loop over all data folders to get the readout frequency and flux pulse amplitude
for tuid in tuid_ssro_200:
    try:
        freq = get_RO_freq(tuid)
        flux_amp = get_flux_pulse_amp(tuid)
        fidelities = [get_raw_ass_fidelity(tuid), get_fid_discr(tuid)]
        
        freq_ro_200.append(freq)
        flux_amp_200.append(flux_amp)
        fid_ass_raw_200.append(fidelities[0])
        fid_discr_200.append(fidelities[1])
    except:
        print(tuid)

In [ ]:
idx_200_resfreq1 = np.argwhere(np.array(freq_ro_200) == 5.1755e9)[:,0]
idx_200_resfreq2 = np.argwhere(np.array(freq_ro_200) == 5.176e9)[:,0]
idx_200_resfreq3 = np.argwhere(np.array(freq_ro_200) == 5.1765e9)[:,0]
idx_200_resfreq4 = np.argwhere(np.array(freq_ro_200) == 5.177e9)[:,0]

fid_ass_raw_200_1 = [fid_ass_raw_200[i] for i in idx_200_resfreq1]
fid_ass_raw_200_2 = [fid_ass_raw_200[i] for i in idx_200_resfreq2]
fid_ass_raw_200_3 = [fid_ass_raw_200[i] for i in idx_200_resfreq3]
fid_ass_raw_200_4 = [fid_ass_raw_200[i] for i in idx_200_resfreq4]

fid_discr_200_1 = [fid_discr_200[i] for i in idx_200_resfreq1]
fid_discr_200_2 = [fid_discr_200[i] for i in idx_200_resfreq2]
fid_discr_200_3 = [fid_discr_200[i] for i in idx_200_resfreq3]
fid_discr_200_4 = [fid_discr_200[i] for i in idx_200_resfreq4]

flux_amp_200_1 = [flux_amp_200[i] for i in idx_200_resfreq1]
flux_amp_200_2 = [flux_amp_200[i] for i in idx_200_resfreq2]
flux_amp_200_3 = [flux_amp_200[i] for i in idx_200_resfreq3]
flux_amp_200_4 = [flux_amp_200[i] for i in idx_200_resfreq4]

In [ ]:
plt.xlabel('Flux Pulse Amplitude (V)')
plt.ylabel('Raw Assignment Fidelity')
plt.title('Readout Amplitude: 200 mV')
plt.xlim(flux_amp_200_1[0], flux_amp_200_1[-1])
plt.plot(flux_amp_200_1, fid_ass_raw_200_1, marker='o', linestyle=' ', color=cmap_blue(0.4), label='5.1755 GHz')
plt.plot(flux_amp_200_2, fid_ass_raw_200_2, marker='o', linestyle=' ', color=cmap_blue(0.6), label='5.176 GHz')
plt.plot(flux_amp_200_3, fid_ass_raw_200_3, marker='o', linestyle=' ', color=cmap_blue(0.8), label='5.1765 GHz')
plt.plot(flux_amp_200_4, fid_ass_raw_200_4, marker='o', linestyle=' ', color=cmap_blue(1.0), label='5.177 GHz')
plt.legend(loc='best', frameon=False)
plt.show()

## 300 mV

In [ ]:
tuid_ssro_300 = get_tuids_containing('SSRO FX8', t_start='20231115-132801', t_stop='20231115-141905')

In [ ]:
freq_ro_300 = []
flux_amp_300 = []

fid_ass_raw_300 = []
fid_discr_300 = []

# Loop over all data folders to get the readout frequency and flux pulse amplitude
for tuid in tuid_ssro_300:
    try:
        freq = get_RO_freq(tuid)
        flux_amp = get_flux_pulse_amp(tuid)
        fidelities = [get_raw_ass_fidelity(tuid), get_fid_discr(tuid)]
        
        freq_ro_300.append(freq)
        flux_amp_300.append(flux_amp)
        fid_ass_raw_300.append(fidelities[0])
        fid_discr_300.append(fidelities[1])
    except:
        print(tuid)

In [ ]:
idx_300_resfreq1 = np.argwhere(np.array(freq_ro_300) == 5.1755e9)[:,0]
idx_300_resfreq2 = np.argwhere(np.array(freq_ro_300) == 5.176e9)[:,0]
idx_300_resfreq3 = np.argwhere(np.array(freq_ro_300) == 5.1765e9)[:,0]
idx_300_resfreq4 = np.argwhere(np.array(freq_ro_300) == 5.177e9)[:,0]

fid_ass_raw_300_1 = [fid_ass_raw_300[i] for i in idx_300_resfreq1]
fid_ass_raw_300_2 = [fid_ass_raw_300[i] for i in idx_300_resfreq2]
fid_ass_raw_300_3 = [fid_ass_raw_300[i] for i in idx_300_resfreq3]
fid_ass_raw_300_4 = [fid_ass_raw_300[i] for i in idx_300_resfreq4]

fid_discr_300_1 = [fid_discr_300[i] for i in idx_300_resfreq1]
fid_discr_300_2 = [fid_discr_300[i] for i in idx_300_resfreq2]
fid_discr_300_3 = [fid_discr_300[i] for i in idx_300_resfreq3]
fid_discr_300_4 = [fid_discr_300[i] for i in idx_300_resfreq4]

flux_amp_300_1 = [flux_amp_300[i] for i in idx_300_resfreq1]
flux_amp_300_2 = [flux_amp_300[i] for i in idx_300_resfreq2]
flux_amp_300_3 = [flux_amp_300[i] for i in idx_300_resfreq3]
flux_amp_300_4 = [flux_amp_300[i] for i in idx_300_resfreq4]

In [ ]:
plt.xlabel('Flux Pulse Amplitude (V)')
plt.ylabel('Raw Assignment Fidelity')
plt.title('Readout Amplitude: 300 mV')
plt.xlim(flux_amp_300_1[0], flux_amp_300_1[-1])
plt.plot(flux_amp_300_1, fid_ass_raw_300_1, marker='o', linestyle=' ', color=cmap_red(0.4), label='5.1755 GHz')
plt.plot(flux_amp_300_2, fid_ass_raw_300_2, marker='o', linestyle=' ', color=cmap_red(0.6), label='5.176 GHz')
plt.plot(flux_amp_300_3, fid_ass_raw_300_3, marker='o', linestyle=' ', color=cmap_red(0.8), label='5.1765 GHz')
plt.plot(flux_amp_300_4, fid_ass_raw_300_4, marker='o', linestyle=' ', color=cmap_red(1.0), label='5.177 GHz')
plt.legend(loc='best', frameon=False)
plt.show()

## 400 mV

In [ ]:
tuid_ssro_400 = get_tuids_containing('SSRO FX8', t_start='20231117-151952', t_stop='20231117-161422')

In [ ]:
freq_ro_400 = []
flux_amp_400 = []

fid_ass_raw_400 = []
fid_discr_400 = []

# Loop over all data folders to get the readout frequency and flux pulse amplitude
for tuid in tuid_ssro_400:
    try:
        freq = get_RO_freq(tuid)
        flux_amp = get_flux_pulse_amp(tuid)
        fidelities = [get_raw_ass_fidelity(tuid), get_fid_discr(tuid)]
        
        freq_ro_400.append(freq)
        flux_amp_400.append(flux_amp)
        fid_ass_raw_400.append(fidelities[0])
        fid_discr_400.append(fidelities[1])
    except:
        print(tuid)

In [ ]:
idx_400_resfreq1 = np.argwhere(np.array(freq_ro_400) == 5.1755e9)[:,0]
idx_400_resfreq2 = np.argwhere(np.array(freq_ro_400) == 5.176e9)[:,0]
idx_400_resfreq3 = np.argwhere(np.array(freq_ro_400) == 5.1765e9)[:,0]
idx_400_resfreq4 = np.argwhere(np.array(freq_ro_400) == 5.177e9)[:,0]

fid_ass_raw_400_1 = [fid_ass_raw_400[i] for i in idx_400_resfreq1]
fid_ass_raw_400_2 = [fid_ass_raw_400[i] for i in idx_400_resfreq2]
fid_ass_raw_400_3 = [fid_ass_raw_400[i] for i in idx_400_resfreq3]
fid_ass_raw_400_4 = [fid_ass_raw_400[i] for i in idx_400_resfreq4]

fid_discr_400_1 = [fid_discr_400[i] for i in idx_400_resfreq1]
fid_discr_400_2 = [fid_discr_400[i] for i in idx_400_resfreq2]
fid_discr_400_3 = [fid_discr_400[i] for i in idx_400_resfreq3]
fid_discr_400_4 = [fid_discr_400[i] for i in idx_400_resfreq4]

flux_amp_400_1 = [flux_amp_400[i] for i in idx_400_resfreq1]
flux_amp_400_2 = [flux_amp_400[i] for i in idx_400_resfreq2]
flux_amp_400_3 = [flux_amp_400[i] for i in idx_400_resfreq3]
flux_amp_400_4 = [flux_amp_400[i] for i in idx_400_resfreq4]

In [ ]:
plt.xlabel('Flux Pulse Amplitude (V)')
plt.ylabel('Raw Assignment Fidelity')
plt.title('Readout Amplitude: 400 mV')
plt.xlim(flux_amp_400_1[0], flux_amp_400_1[-1])
plt.plot(flux_amp_400_1, fid_ass_raw_400_1, marker='o', linestyle=' ', color=cmap_orange(0.4), label='5.1755 GHz')
plt.plot(flux_amp_400_2, fid_ass_raw_400_2, marker='o', linestyle=' ', color=cmap_orange(0.6), label='5.176 GHz')
plt.plot(flux_amp_400_3, fid_ass_raw_400_3, marker='o', linestyle=' ', color=cmap_orange(0.8), label='5.1765 GHz')
plt.plot(flux_amp_400_4, fid_ass_raw_400_4, marker='o', linestyle=' ', color=cmap_orange(1.0), label='5.177 GHz')
plt.legend(loc='best', frameon=False)
plt.show()

## Plotting

In [ ]:
phiext_100 = []
phiext_150 = []
phiext_200 = []
phiext_300 = []
phiext_400 = []

for i in flux_amp_100_2:
    phiext_100.append((0.2/0.1155)*i + 0.5)
for i in flux_amp_150_2:
    phiext_150.append((0.2/0.1155)*i + 0.5)
for i in flux_amp_200_2:
    phiext_200.append((0.2/0.1155)*i + 0.5)
for i in flux_amp_300_2:
    phiext_300.append((0.2/0.1155)*i + 0.5)
for i in flux_amp_400_2:
    phiext_400.append((0.2/0.1155)*i + 0.5)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax

tick_fs = 9
label_fs = 10
lw = 1.5

color_intensity = 0.8
color_100=cmap_purple(color_intensity)
color_150=cmap_green(color_intensity)
color_200=cmap_blue(color_intensity)
color_300=cmap_red(color_intensity)
color_400=cmap_orange(color_intensity)

a, = ax1.plot(phiext_100, fid_ass_raw_100_2, color=color_100, linestyle='', marker='.', markersize=6, label=r"100 mV")
b, = ax1.plot(phiext_150, fid_ass_raw_150_2, color=color_150, linestyle='', marker='.', markersize=6, label=r"150 mV")
c, = ax1.plot(phiext_200, fid_ass_raw_200_2, color=color_200, linestyle='', marker='.', markersize=6, label=r"200 mV")
d, = ax1.plot(phiext_300, fid_ass_raw_300_2, color=color_300, linestyle='', marker='.', markersize=6, label=r"300 mV")
e, = ax1.plot(phiext_400, fid_ass_raw_400_2, color=color_400, linestyle='', marker='.', markersize=6, label=r"400 mV")
ax1.tick_params(axis='y', pad=3, labelsize=tick_fs)
ax1.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax1.set_xlim(phiext_100[0], phiext_100[-1])
ax1.set_ylim(0.5, 0.95)
ax1.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=label_fs)
ax1.set_ylabel(r"Raw Assign. Fidelity", fontsize=label_fs, labelpad=2)

l1 = ax1.legend([a, b], [r"100 mV", r"150 mV"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=-0.3, frameon=False, fontsize = tick_fs, labelspacing=0.3, 
               loc='upper left', bbox_to_anchor=(0.38, 1.05))
l2 = ax1.legend([c, d, e], [r"200 mV", r"300 mV", r"400 mV"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=-0.3, frameon=False, fontsize = tick_fs, labelspacing=0.3, 
               loc='upper left', bbox_to_anchor=(0.66, 1.05))
ax1.add_artist(l1)
ax1.add_artist(l2)

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax

tick_fs = 9
label_fs = 10
lw = 1.5

color_intensity = 0.8
color_100=cmap_purple(color_intensity)
color_150=cmap_green(color_intensity)
color_200=cmap_blue(color_intensity)
color_300=cmap_red(color_intensity)
color_400=cmap_orange(color_intensity)

a, = ax1.plot(phiext_100, fid_ass_raw_100_2, color=color_100, linestyle='', marker='.', markersize=6, label=r"100 mV")
b, = ax1.plot(phiext_150, fid_ass_raw_150_2, color=color_150, linestyle='', marker='.', markersize=6, label=r"150 mV")
c, = ax1.plot(phiext_200, fid_ass_raw_200_2, color=color_200, linestyle='', marker='.', markersize=6, label=r"200 mV")
d, = ax1.plot(phiext_300, fid_ass_raw_300_2, color=color_300, linestyle='', marker='.', markersize=6, label=r"300 mV")
e, = ax1.plot(phiext_400, fid_ass_raw_400_2, color=color_400, linestyle='', marker='.', markersize=6, label=r"400 mV")
ax1.tick_params(axis='y', pad=3, labelsize=tick_fs)
ax1.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax1.set_xlim(phiext_100[0], phiext_100[-1])
ax1.set_ylim(0.5, 0.95)
ax1.set_xlabel(r"$\Phi_{ext}/\Phi_0$", fontsize=label_fs)
ax1.set_ylabel(r"Assign. Fidelity", fontsize=label_fs, labelpad=2)
l1 = ax1.legend([a, b], [r"$\bar{n}$ = 3.3", r"$\bar{n}$ = 7.3"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=-0.5, frameon=False, fontsize = tick_fs, labelspacing=0.3, 
               loc='upper left', bbox_to_anchor=(0.38, 1.05))
l2 = ax1.legend([c, d, e], [r"$\bar{n}$ = 13.0", r"$\bar{n}$ = 29.3", r"$\bar{n}$ = 52.1"],
               handler_map={tuple: HandlerTuple(ndivide=None)}, handletextpad=-0.5, frameon=False, fontsize = tick_fs, labelspacing=0.3, 
               loc='upper left', bbox_to_anchor=(0.63, 1.05))
ax1.add_artist(l1)
ax1.add_artist(l2)

# Fidelity vs RO frequency at SS

In [ ]:
tuid_nofp_100 = get_tuids_containing('SSRO FX8', t_start='20231117-165915', t_stop='20231117-170256')
tuid_nofp_150 = get_tuids_containing('SSRO FX8', t_start='20231117-170442', t_stop='20231117-170802')
tuid_nofp_200 = get_tuids_containing('SSRO FX8', t_start='20231117-170918', t_stop='20231117-171222')
tuid_nofp_300 = get_tuids_containing('SSRO FX8', t_start='20231117-171426', t_stop='20231117-171728')
tuid_nofp_400 = get_tuids_containing('SSRO FX8', t_start='20231117-171801', t_stop='20231117-172058')

In [ ]:
freq_ro_100 = []
freq_ro_150 = []
freq_ro_200 = []
freq_ro_300 = []
freq_ro_400 = []

fid_ass_raw_nofp_100 = []
fid_ass_raw_nofp_150 = []
fid_ass_raw_nofp_200 = []
fid_ass_raw_nofp_300 = []
fid_ass_raw_nofp_400 = []

# Loop over all data folders to find all the readout frequencies
for i,tuid in enumerate(tuid_nofp_100):
    try:
        freq_ro_100.append(get_RO_freq(tuid))
        fid_ass_raw_nofp_100.append(get_raw_ass_fidelity(tuid))
    except:
        print(tuid)
        
for i,tuid in enumerate(tuid_nofp_150):
    try:
        freq_ro_150.append(get_RO_freq(tuid))
        fid_ass_raw_nofp_150.append(get_raw_ass_fidelity(tuid))
    except:
        print(tuid)
        
for i,tuid in enumerate(tuid_nofp_200):
    try:
        freq_ro_200.append(get_RO_freq(tuid))
        fid_ass_raw_nofp_200.append(get_raw_ass_fidelity(tuid))
    except:
        print(tuid)
        
for i,tuid in enumerate(tuid_nofp_300):
    try:
        freq_ro_300.append(get_RO_freq(tuid))
        fid_ass_raw_nofp_300.append(get_raw_ass_fidelity(tuid))
    except:
        print(tuid)
        
for i,tuid in enumerate(tuid_nofp_400):
    try:
        freq_ro_400.append(get_RO_freq(tuid))
        fid_ass_raw_nofp_400.append(get_raw_ass_fidelity(tuid))
    except:
        print(tuid)

## Plotting

In [ ]:
plt.xlabel('Readout Frequency (GHz)')
plt.ylabel('Raw Assignment Fidelity')
plt.xlim(5.174, 5.1775)
plt.ylim(0.5, 0.8)
plt.plot(np.array(freq_ro_100)*10**(-9), fid_ass_raw_nofp_100, marker='o', linestyle=' ', color=cmap_purple(0.8), label='100 mV')
plt.plot(np.array(freq_ro_150)*10**(-9), fid_ass_raw_nofp_150, marker='o', linestyle=' ', color=cmap_green(0.8), label='150 mV')
plt.plot(np.array(freq_ro_200)*10**(-9), fid_ass_raw_nofp_200, marker='o', linestyle=' ', color=cmap_blue(0.8), label='200 mV')
plt.plot(np.array(freq_ro_300)*10**(-9), fid_ass_raw_nofp_300, marker='o', linestyle=' ', color=cmap_red(0.8), label='300 mV')
plt.plot(np.array(freq_ro_400)*10**(-9), fid_ass_raw_nofp_400, marker='o', linestyle=' ', color=cmap_orange(0.8), label='400 mV')
plt.legend()
plt.show()

# Fidelity vs Integration Time (variable $\Delta \Phi_{ext}/\Phi_{0}$)

In [ ]:
tuid_static_100 = get_tuids_containing('SSRO FX8', t_start='20231118-205254', t_stop='20231118-210007')
tuid_fpa_100 = get_tuids_containing('SSRO FX8', t_start='20231119-023751', t_stop='20231119-024513')

tuid_static_150 = get_tuids_containing('SSRO FX8', t_start='20231118-210336', t_stop='20231118-211104')
tuid_fpa_150 = get_tuids_containing('SSRO FX8', t_start='20231119-024539', t_stop='20231119-025321')

tuid_static_200 = get_tuids_containing('SSRO FX8', t_start='20231118-211649', t_stop='20231118-212421')
tuid_fpa_200 = get_tuids_containing('SSRO FX8', t_start='20231119-025346', t_stop='20231119-030126')

tuid_static_300 = get_tuids_containing('SSRO FX8', t_start='20231118-213438', t_stop='20231118-214159')
tuid_fpa_300 = get_tuids_containing('SSRO FX8', t_start='20231119-030153', t_stop='20231119-030934')

tuid_static_400 = get_tuids_containing('SSRO FX8', t_start='20231118-215036', t_stop='20231118-215757')
tuid_fpa_400 = get_tuids_containing('SSRO FX8', t_start='20231119-031000', t_stop='20231119-031730')

## 100 mV
with FPA readout, $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1697

In [ ]:
fid_ass_raw_static_100 = []
fid_discr_static_100 = []
int_times_static_100 = []

for tuid in tuid_static_100:
    fid_ass_raw_static_100.append(get_raw_ass_fidelity(tuid))
    fid_discr_static_100.append(get_fid_discr(tuid))
    int_times_static_100.append(get_integration_time(tuid)*10**(6))

fid_ass_raw_fpa_100 = []
fid_discr_fpa_100 = []
int_times_fpa_100 = []

for tuid in tuid_fpa_100:
    fid_ass_raw_fpa_100.append(get_raw_ass_fidelity(tuid))
    fid_discr_fpa_100.append(get_fid_discr(tuid))
    int_times_fpa_100.append(get_integration_time(tuid)*10**(6))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 100 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_100[0], int_times_static_100[-1])
plt.plot(int_times_static_100, fid_ass_raw_static_100, marker='o', linestyle=' ', color=cmap_purple(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_100, fid_ass_raw_fpa_100, marker='o', linestyle=' ', color=cmap_purple(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_100, fid_discr_static_100, marker='P', linestyle=' ', color=cmap_purple(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_100, fid_discr_fpa_100, marker='P', linestyle=' ', color=cmap_purple(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 150 mV
with FPA readout, $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1697

In [ ]:
fid_ass_raw_static_150 = []
fid_discr_static_150 = []
int_times_static_150 = []

for tuid in tuid_static_150:
    fid_ass_raw_static_150.append(get_raw_ass_fidelity(tuid))
    fid_discr_static_150.append(get_fid_discr(tuid))
    int_times_static_150.append(get_integration_time(tuid)*10**(6))

fid_ass_raw_fpa_150 = []
fid_discr_fpa_150 = []
int_times_fpa_150 = []

for tuid in tuid_fpa_150:
    fid_ass_raw_fpa_150.append(get_raw_ass_fidelity(tuid))
    fid_discr_fpa_150.append(get_fid_discr(tuid))
    int_times_fpa_150.append(get_integration_time(tuid)*10**(6))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 150 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_150[0], int_times_static_150[-1])
plt.plot(int_times_static_150, fid_ass_raw_static_150, marker='o', linestyle=' ', color=cmap_green(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_150, fid_ass_raw_fpa_150, marker='o', linestyle=' ', color=cmap_green(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_150, fid_discr_static_150, marker='P', linestyle=' ', color=cmap_green(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_150, fid_discr_fpa_150, marker='P', linestyle=' ', color=cmap_green(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 200 mV
with FPA readout, $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1723

In [ ]:
fid_ass_raw_static_200 = []
fid_discr_static_200 = []
int_times_static_200 = []

for tuid in tuid_static_200:
    fid_ass_raw_static_200.append(get_raw_ass_fidelity(tuid))
    fid_discr_static_200.append(get_fid_discr(tuid))
    int_times_static_200.append(get_integration_time(tuid)*10**(6))

fid_ass_raw_fpa_200 = []
fid_discr_fpa_200 = []
int_times_fpa_200 = []

for tuid in tuid_fpa_200:
    fid_ass_raw_fpa_200.append(get_raw_ass_fidelity(tuid))
    fid_discr_fpa_200.append(get_fid_discr(tuid))
    int_times_fpa_200.append(get_integration_time(tuid)*10**(6))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 200 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_200[0], int_times_static_200[-1])
plt.plot(int_times_static_200, fid_ass_raw_static_200, marker='o', linestyle=' ', color=cmap_blue(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_200, fid_ass_raw_fpa_200, marker='o', linestyle=' ', color=cmap_blue(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_200, fid_discr_static_200, marker='P', linestyle=' ', color=cmap_blue(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_200, fid_discr_fpa_200, marker='P', linestyle=' ', color=cmap_blue(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 300 mV
with FPA readout, $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1593

In [ ]:
fid_ass_raw_static_300 = []
fid_discr_static_300 = []
int_times_static_300 = []

for tuid in tuid_static_300:
    fid_ass_raw_static_300.append(get_raw_ass_fidelity(tuid))
    fid_discr_static_300.append(get_fid_discr(tuid))
    int_times_static_300.append(get_integration_time(tuid)*10**(6))

fid_ass_raw_fpa_300 = []
fid_discr_fpa_300 = []
int_times_fpa_300 = []

for tuid in tuid_fpa_300:
    fid_ass_raw_fpa_300.append(get_raw_ass_fidelity(tuid))
    fid_discr_fpa_300.append(get_fid_discr(tuid))
    int_times_fpa_300.append(get_integration_time(tuid)*10**(6))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 300 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_300[0], int_times_static_300[-1])
plt.plot(int_times_static_300, fid_ass_raw_static_300, marker='o', linestyle=' ', color=cmap_red(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_300, fid_ass_raw_fpa_300, marker='o', linestyle=' ', color=cmap_red(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_300, fid_discr_static_300, marker='P', linestyle=' ', color=cmap_red(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_300, fid_discr_fpa_300, marker='P', linestyle=' ', color=cmap_red(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 400 mV
with FPA readout, $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1567

In [ ]:
fid_ass_raw_static_400 = []
fid_discr_static_400 = []
int_times_static_400 = []

for tuid in tuid_static_400:
    fid_ass_raw_static_400.append(get_raw_ass_fidelity(tuid))
    fid_discr_static_400.append(get_fid_discr(tuid))
    int_times_static_400.append(get_integration_time(tuid)*10**(6))

fid_ass_raw_fpa_400 = []
fid_discr_fpa_400 = []
int_times_fpa_400 = []

for tuid in tuid_fpa_400:
    fid_ass_raw_fpa_400.append(get_raw_ass_fidelity(tuid))
    fid_discr_fpa_400.append(get_fid_discr(tuid))
    int_times_fpa_400.append(get_integration_time(tuid)*10**(6))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 400 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_400[0], int_times_static_400[-1])
plt.plot(int_times_static_400, fid_ass_raw_static_400, marker='o', linestyle=' ', color=cmap_orange(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_400, fid_ass_raw_fpa_400, marker='o', linestyle=' ', color=cmap_orange(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_400, fid_discr_static_400, marker='P', linestyle=' ', color=cmap_orange(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_400, fid_discr_fpa_400, marker='P', linestyle=' ', color=cmap_orange(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

# Fidelity vs Integration Time for $\Delta \Phi_{ext}/\Phi_{0}$ = 0.1567
integration times only up to 400 ns

In [ ]:
tuid_static_100_short = get_tuids_containing('SSRO FX8', t_start='20231201-175030', t_stop='20231201-180335')
tuid_fpa_100_short = get_tuids_containing('SSRO FX8', t_start='20231201-190020', t_stop='20231201-191329')

tuid_static_150_short = get_tuids_containing('SSRO FX8', t_start='20231201-180353', t_stop='20231201-181625')
tuid_fpa_150_short = get_tuids_containing('SSRO FX8', t_start='20231201-191358', t_stop='20231201-192628')

tuid_static_200_short = get_tuids_containing('SSRO FX8', t_start='20231201-181655', t_stop='20231201-183247')
tuid_fpa_200_short = get_tuids_containing('SSRO FX8', t_start='20231201-192705', t_stop='20231201-194046')

tuid_static_300_short = get_tuids_containing('SSRO FX8', t_start='20231201-183330', t_stop='20231201-184617')
tuid_fpa_300_short = get_tuids_containing('SSRO FX8', t_start='20231201-194103', t_stop='20231201-195726')

tuid_static_400_short = get_tuids_containing('SSRO FX8', t_start='20231201-184646', t_stop='20231201-190000')
tuid_fpa_400_short = get_tuids_containing('SSRO FX8', t_start='20231201-195756', t_stop='20231201-201211')

## 100 mV

In [ ]:
fid_ass_raw_static_100_short = []
int_times_static_100_short = []
fid_discr_static_100_short = []

for i in tuid_static_100_short:
    fid_ass_raw_static_100_short.append(get_raw_ass_fidelity(i))
    int_times_static_100_short.append(get_integration_time(i)*10**(9))
    fid_discr_static_100_short.append(get_fid_discr(i))


fid_ass_raw_fpa_100_short = []
int_times_fpa_100_short = []
fid_discr_fpa_100_short = []

for i in tuid_fpa_100_short:
    fid_ass_raw_fpa_100_short.append(get_raw_ass_fidelity(i))
    int_times_fpa_100_short.append(get_integration_time(i)*10**(9))
    fid_discr_fpa_100_short.append(get_fid_discr(i))

In [ ]:
plt.xlabel('Integration Time (ns)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 100 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_100_short[0], int_times_static_100_short[-1])
plt.plot(int_times_static_100_short, fid_ass_raw_static_100_short, marker='o', linestyle=' ', color=cmap_purple(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_100_short, fid_ass_raw_fpa_100_short, marker='o', linestyle=' ', color=cmap_purple(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_100_short, fid_discr_static_100_short, marker='P', linestyle=' ', color=cmap_purple(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_100_short, fid_discr_fpa_100_short, marker='P', linestyle=' ', color=cmap_purple(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 150 mV

In [ ]:
fid_ass_raw_static_150_short = []
int_times_static_150_short = []
fid_discr_static_150_short = []

for i in tuid_static_150_short:
    fid_ass_raw_static_150_short.append(get_raw_ass_fidelity(i))
    int_times_static_150_short.append(get_integration_time(i)*10**(9))
    fid_discr_static_150_short.append(get_fid_discr(i))


fid_ass_raw_fpa_150_short = []
int_times_fpa_150_short = []
fid_discr_fpa_150_short = []

for i in tuid_fpa_150_short:
    fid_ass_raw_fpa_150_short.append(get_raw_ass_fidelity(i))
    int_times_fpa_150_short.append(get_integration_time(i)*10**(9))
    fid_discr_fpa_150_short.append(get_fid_discr(i))

In [ ]:
plt.xlabel('Integration Time (ns)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 150 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_150_short[0], int_times_static_150_short[-1])
plt.plot(int_times_static_150_short, fid_ass_raw_static_150_short, marker='o', linestyle=' ', color=cmap_green(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_150_short, fid_ass_raw_fpa_150_short, marker='o', linestyle=' ', color=cmap_green(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_150_short, fid_discr_static_150_short, marker='P', linestyle=' ', color=cmap_green(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_150_short, fid_discr_fpa_150_short, marker='P', linestyle=' ', color=cmap_green(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 200 mV

In [ ]:
fid_ass_raw_static_200_short = []
int_times_static_200_short = []
fid_discr_static_200_short = []

for i in tuid_static_200_short:
    try:
        int_times_static_200_short.append(get_integration_time(i)*10**(9))
        fid_ass_raw_static_200_short.append(get_raw_ass_fidelity(i))
        fid_discr_static_200_short.append(get_fid_discr(i))
    except:
        print(tuid)


fid_ass_raw_fpa_200_short = []
int_times_fpa_200_short = []
fid_discr_fpa_200_short = []

for i in tuid_fpa_200_short:
    fid_ass_raw_fpa_200_short.append(get_raw_ass_fidelity(i))
    int_times_fpa_200_short.append(get_integration_time(i)*10**(9))
    fid_discr_fpa_200_short.append(get_fid_discr(i))

In [ ]:
plt.xlabel('Integration Time (ns)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 200 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_200_short[0], int_times_static_200_short[-1])
plt.plot(int_times_static_200_short, fid_ass_raw_static_200_short, marker='o', linestyle=' ', color=cmap_blue(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_200_short, fid_ass_raw_fpa_200_short, marker='o', linestyle=' ', color=cmap_blue(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_200_short, fid_discr_static_200_short, marker='P', linestyle=' ', color=cmap_blue(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_200_short, fid_discr_fpa_200_short, marker='P', linestyle=' ', color=cmap_blue(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 300 mV

In [ ]:
fid_ass_raw_static_300_short = []
int_times_static_300_short = []
fid_discr_static_300_short = []

for i in tuid_static_300_short:
    fid_ass_raw_static_300_short.append(get_raw_ass_fidelity(i))
    int_times_static_300_short.append(get_integration_time(i)*10**(9))
    fid_discr_static_300_short.append(get_fid_discr(i))


fid_ass_raw_fpa_300_short = []
int_times_fpa_300_short = []
fid_discr_fpa_300_short = []

for i in tuid_fpa_300_short:
    fid_ass_raw_fpa_300_short.append(get_raw_ass_fidelity(i))
    int_times_fpa_300_short.append(get_integration_time(i)*10**(9))
    fid_discr_fpa_300_short.append(get_fid_discr(i))

In [ ]:
plt.xlabel('Integration Time (ns)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 300 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_300_short[0], int_times_static_300_short[-1])
plt.plot(int_times_static_300_short, fid_ass_raw_static_300_short, marker='o', linestyle=' ', color=cmap_red(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_300_short, fid_ass_raw_fpa_300_short, marker='o', linestyle=' ', color=cmap_red(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_300_short, fid_discr_static_300_short, marker='P', linestyle=' ', color=cmap_red(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_300_short, fid_discr_fpa_300_short, marker='P', linestyle=' ', color=cmap_red(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

## 400 mV

In [ ]:
fid_ass_raw_static_400_short = []
int_times_static_400_short = []
fid_discr_static_400_short = []

for i in tuid_static_400_short:
    try:
        int_times_static_400_short.append(get_integration_time(i)*10**(9))
        fid_ass_raw_static_400_short.append(get_raw_ass_fidelity(i))
        fid_discr_static_400_short.append(get_fid_discr(i))
    except:
        print(i)

fid_ass_raw_fpa_400_short = []
int_times_fpa_400_short = []
fid_discr_fpa_400_short = []

for i in tuid_fpa_400_short:
    fid_ass_raw_fpa_400_short.append(get_raw_ass_fidelity(i))
    int_times_fpa_400_short.append(get_integration_time(i)*10**(9))
    fid_discr_fpa_400_short.append(get_fid_discr(i))

In [ ]:
plt.xlabel('Integration Time (us)')
plt.ylabel('Fidelity')
plt.title('Readout Amplitude: 400 mV')
plt.ylim(0.5, 1)
plt.xlim(int_times_static_400_short[0], int_times_static_400_short[-1])
plt.plot(int_times_static_400_short, fid_ass_raw_static_400_short, marker='o', linestyle=' ', color=cmap_orange(0.6), label=r'$F_{assign}$, SS')
plt.plot(int_times_fpa_400_short, fid_ass_raw_fpa_400_short, marker='o', linestyle=' ', color=cmap_orange(0.8), label=r'$F_{assign}$, FPA')
plt.plot(int_times_static_400_short, fid_discr_static_400_short, marker='P', linestyle=' ', color=cmap_orange(0.6), label=r'$F_{SNR}$, SS')
plt.plot(int_times_fpa_400_short, fid_discr_fpa_400_short, marker='P', linestyle=' ', color=cmap_orange(0.8), label=r'$F_{SNR}$, FPA')
plt.legend(loc='best', frameon=False)
plt.show()

In [ ]:
snr_paperdef_static_400_short = []
error_paperdef_static_400_short = []

for i in tuid_static_400_short:
    try:
        _snr, _error = get_paperdef_snr_error(i)
        snr_paperdef_static_400_short.append(_snr)
        error_paperdef_static_400_short.append(_error)
    except:
        print(i)

snr_paperdef_fpa_400_short = []
error_paperdef_fpa_400_short = []

for i in tuid_fpa_400_short:
    _snr, _error = get_paperdef_snr_error(i)
    snr_paperdef_fpa_400_short.append(_snr)
    error_paperdef_fpa_400_short.append(_error)

In [ ]:
font = {'size'   : 16}

mpl.rc('font', **font)

plt.xlabel('Integration Time (ns)')
plt.ylabel('SNR Limited Error')
plt.xlim(100, 360)
plt.yscale('log')
plt.ylim(0.0005,1)
raw_static = plt.plot(int_times_static_400_short, 1-np.array(fid_ass_raw_static_400_short), marker='.', markersize=10, linestyle=' ', color=cmap_red(0.6), label=r'$F_{assign}$, SS')
raw_fpa = plt.plot(int_times_fpa_400_short, 1-np.array(fid_ass_raw_fpa_400_short), marker='.', markersize=10, linestyle=' ', color=cmap_blue(0.7), label=r'$F_{assign}$, FPA')
snr_lim_static = plt.plot(int_times_static_400_short, error_paperdef_static_400_short, marker='P', linestyle=' ', color=cmap_red(0.8), label=r'$F_{SNR}$, SS')
snr_lim_fpa = plt.plot(int_times_fpa_400_short, error_paperdef_fpa_400_short, marker='P', linestyle=' ', color=cmap_blue(0.9), label=r'$F_{SNR}$, SS')
plt.annotate('RO Amplitude: 400 mV', xy=(0.47, 0.92), xycoords='axes fraction')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=250)

fig.subplots_adjust(bottom = 0.16, top=0.99, left=0.19, right=0.9, wspace=0.4, hspace=0.3)

fig.set_size_inches(7/2, 2)
ax1 = ax

tick_fs = 9
label_fs = 10
lw = 1.5


raw_static, = ax1.plot(int_times_static_400_short, 1-np.array(fid_ass_raw_static_400_short), color=cmap_red(0.7), linestyle='', marker='.', markersize=6, label=r'SS, Assign.(raw)')
raw_fpa, = ax1.plot(int_times_fpa_400_short, 1-np.array(fid_ass_raw_fpa_400_short), color=cmap_blue(0.7), linestyle='', marker='.', markersize=6, label=r'FPA, Assign.(raw)')
snr_lim_static, = ax1.plot(int_times_static_400_short, error_paperdef_static_400_short, color=cmap_red(0.9), linestyle='', marker='+', markersize=6, label=r'SS, SNR-limited')
snr_lim_fpa, = ax1.plot(int_times_fpa_400_short, error_paperdef_fpa_400_short, color=cmap_blue(0.9), linestyle='', marker='+', markersize=6, label=r'FPA, SNR-limited')
ax1.tick_params(axis='y', pad=3, labelsize=tick_fs)
ax1.tick_params(axis='x', pad=3, labelsize=tick_fs)
ax1.set_yscale('log')
ax1.set_xlim(100, 360)
ax1.set_ylim(0.0005, 1)
ax1.set_xlabel('Integration Time (ns)', fontsize=label_fs)
ax1.set_ylabel('Readout Error', fontsize=label_fs, labelpad=2)
ax1.legend(handletextpad=0.25, frameon=False, handlelength = 1.0, fontsize = tick_fs, labelspacing=0.3, 
               loc='lower left', bbox_to_anchor=(-0.03, -0.05))

In [ ]:
freq_static = []
freq_fpa = []

for i in tuid_static_400_short:
    f = get_RO_freq(i)
    freq_static.append(f)
    
for i in tuid_fpa_400_short:
    f = get_RO_freq(i)
    freq_fpa.append(f)

In [ ]:
import pandas as pd
data = {'RawErrorSS': 1-np.array(fid_ass_raw_static_400_short),
        'RawErrorFPA': 1-np.array(fid_ass_raw_fpa_400_short),
        'DiscrErrorSS': error_paperdef_static_400_short,
        'DiscrErrorFPA': error_paperdef_fpa_400_short,
        'TimesSS': int_times_static_400_short,
        'TimesFPA': int_times_fpa_400_short,
        'ROFreqSS': freq_static,
        'ROFreqFPA': freq_fpa,}

# df = pd.DataFrame.from_dict(data, orient='index')
# df = df.transpose()
# df.to_csv('bestRO.csv')